In [94]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm
from sklearn.preprocessing import OrdinalEncoder

In [95]:
path = "D:/мисис/питон-1/MISIS_2022/season_2/data/athletes_sochi.txt"
dfs = pd.read_csv(path).dropna()

In [124]:
dfs 

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country,BMI,code,codeName,codeSport
0,17,1996-04-12,Male,1.72,Aaron,68.0,0,0,0,0,Freestyle Skiing,United States,22.985398,0.0,0.0,0.0
1,27,1986-05-14,Male,1.85,Aaron,85.0,0,0,0,0,Snowboard,Italy,24.835646,0.0,0.0,0.0
2,21,1992-06-30,Male,1.78,Abzal,68.0,0,0,0,0,Short Track,Kazakhstan,21.461937,1.0,1.0,1.0
4,21,1992-07-30,Male,1.86,Adam,82.0,0,0,0,0,Alpine Skiing,New Zealand,23.702162,2.0,2.0,2.0
5,21,1992-12-18,Male,1.75,Adam,57.0,0,0,0,0,Nordic Combined,Poland,18.612245,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga,98.0,0,0,0,0,Ice Hockey,Slovenia,26.309431,1434.0,1434.0,1434.0
2853,31,1982-12-05,Female,1.70,Zina,60.0,0,0,0,0,Biathlon,Canada,20.761246,1435.0,1435.0,1435.0
2854,28,1985-06-14,Female,1.68,Zoe,65.0,0,0,0,0,Snowboard,Great Britain,23.030045,1436.0,1436.0,1436.0
2856,22,1991-03-01,Male,1.76,Zongyang,68.0,0,0,1,1,Freestyle Skiing,China,21.952479,1437.0,1437.0,1437.0


In [125]:
def getName(df, col_name):
    df = df['name'].str.split(' ', expand=True)[0]
    df['name'] = df[0]
    return df

In [126]:
def codeValue(df, col_name):
    ord_enc = OrdinalEncoder()
    dfs['code'] = ord_enc.fit_transform(dfs[["name"]])
    return dfs['code']

In [138]:
def accuracy(Y_test, Y_test_predicted):
    [tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
    confusion_matrix(Y_test, Y_test_predicted)
    accuracy = (tn + tp) / (tp + fp + tn + fn)
    print("Accuracy: ", accuracy)
    return accuracy

In [127]:
dfs["BMI"] = dfs["weight"] / (dfs["height"]**2)
dfs["name"] = getName(dfs, "name")
dfs["codeName"] = codeValue(dfs, "name") #Вроде, чуть больше стало..
dfs["codeSport"] = codeValue(dfs, "code") #Не изменило особо
alldata = dfs[ ['weight', 'height', 'BMI', 'gender', 'codeName']]
alldata = alldata.sample(frac=1)

allinput = alldata[ ['weight', 'height', 'BMI', 'codeName'] ]
alloutput = alldata["gender"].apply(lambda x: int(x == "Male"))



In [128]:
allinput

,weight,height,BMI,codeName
906,74.0,1.85,21.621622,425.0
759,48.0,1.57,19.473407,347.0
2313,74.0,1.84,21.857278,1150.0
2646,88.0,1.80,27.160494,1313.0
1216,58.0,1.65,21.303949,602.0
...,...,...,...,...
368,95.0,2.00,23.750000,149.0
1446,79.0,1.80,24.382716,728.0
2013,68.0,1.77,21.705129,1003.0
136,65.0,1.72,21.971336,48.0


In [129]:
alloutput

906     1
759     0
2313    1
2646    1
1216    0
       ..
368     1
1446    1
2013    1
136     1
585     1
Name: gender, Length: 2479, dtype: int64

In [130]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.2)

In [131]:
Y_train

101     1
934     1
442     1
1296    0
1368    0
       ..
1357    0
1477    0
488     0
968     1
85      1
Name: gender, Length: 1983, dtype: int64

In [132]:
model = linear_model.LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [133]:
model.coef_

array([[ 3.76061507e-01,  1.60280235e-01, -8.08466583e-01,
         1.95092037e-04]])

In [134]:
Y_test_predicted = model.predict(X_test)

In [139]:
accuracy(Y_test, Y_test_predicted)


Accuracy:  0.8810483870967742


0.8810483870967742